In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "utility"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMol
using StaticArrays
using Rotations
using Distances
using JLD2

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [17]:
folder = "../../Data/hpc_out/rwm_wip_2_6r7m/"
mol_type = "6r7m"
#folder = "../../Data/collected_simulation_results/rwm_op_2_6r7m/"

id_index = 1
separator = "_"
max = 1
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        max = max < parse(Int, split(file, separator)[id_index]) ? parse(Int, split(file, separator)[id_index]) : max
    end
end

acceptance_rates = Vector{Float64}([])
min_Es = [Inf for _ in 1:max]
evaluation_strings = ["" for _ in 1:max]
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        try
            @load "$folder$file" input output
            # Crude test to exclude runs that remained dispersed
            if maximum(output["Es"]) - minimum(output["Es"]) < 10.0# || input["T"] > 1.5
                println("E_diff = $(maximum(output["Es"]) - minimum(output["Es"]))")
                continue
            end
            
            id = parse(Int, split(file, separator)[id_index])
            exp_template_centers = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["template_centers"]
            exp_state = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["state"]
            theta = MorphoMol.Utilities.average_offset_distance(exp_template_centers, input["template_centers"], exp_state, output["states"][argmin(output["Es"])])
            string = "$(id): $(input["T"]) | $(output["αs"][end]) | $(output["Es"][1]), $(minimum(output["Es"])), $(output["Es"][end]) | $(theta)"
            push!(acceptance_rates, output["αs"][end])
            evaluation_strings[id] = string
            min_Es[id] = minimum(output["Es"])
        catch e
            println(e)
            println("Error in $file")
        end
    end
end

JLD2.InvalidDataException("Did not find a Superblock.")
Error in 29_rwm_wip_2_6r7m.jld2
JLD2.InvalidDataException("Did not find a Superblock.")
Error in 33_rwm_wip_2_6r7m.jld2
JLD2.InvalidDataException("Did not find a Superblock.")
Error in 61_rwm_wip_2_6r7m.jld2
JLD2.InvalidDataException("Did not find a Superblock.")
Error in 75_rwm_wip_2_6r7m.jld2
JLD2.InvalidDataException("Did not find a Superblock.")
Error in 79_rwm_wip_2_6r7m.jld2


In [18]:
println("Minimal configuration id: $(argmin(min_Es)) | Average acceptance rate: $(sum(acceptance_rates) / length(acceptance_rates))")

Minimal configuration id: 13 | Average acceptance rate: 0.20323937834473327


In [19]:
for elem in evaluation_strings
    println(elem)
end






6: 1.5 | 0.30323434 | 8219.478396659431, 8176.156845673511, 8176.156845673511 | 4.4166541550122975





12: 1.5 | 0.5089508 | 8219.388947472613, 8178.256220757711, 8178.572304306344 | 23.794316351374288
13: 1.5 | 0.06352829 | 8219.448439055137, 8160.421396014614, 8160.421396014614 | 6.010199297182606
14: 1.5 | 0.2751961 | 8219.558326291683, 8176.619341855061, 8184.90244946654 | 21.969989257596012



18: 1.5 | 0.12492689 | 8219.548324381907, 8185.954350713405, 8185.954350713405 | 34.147022242277785
19: 1.5 | 0.089940585 | 8219.508448255978, 8183.493008756169, 8186.543264864343 | 34.01732075202291
20: 1.5 | 0.1389507 | 8219.468469934762, 8173.547228218821, 8173.547228218821 | 8.635654186816504



24: 1.5 | 0.090319075 | 8219.5583086449, 8168.365968619133, 8174.332748201366 | 14.408361125846792

26: 1.5 | 0.3690562 | 8219.30920037324, 8177.316547684329, 8177.316547684329 | 19.204371465706682
27: 1.5 | 0.064120024 | 8219.528328583172, 8180.980799473264, 8181.245015840412 | 31.69027073

In [48]:
for file in readdir(folder)
    try
        @load "$folder$file" input output
        mindex = argmin(output["Es"])
        state = output["states"][mindex]
        id = parse(Int, split(file, "_")[id_index])
        radii = collect(Base.Iterators.flatten([input["template_radii"] for _ in 1:input["n_mol"]]))
        MorphoMol.Utilities.state_to_poly(MorphoMol.Utilities.get_flat_realization(state, input["template_centers"]), radii, "$(folder)$(id)", input["n_mol"], length(input["template_radii"]))
    catch e
        println(e)
        continue
    end
end

JLD2.InvalidDataException("Did not find a Superblock.")
JLD2.InvalidDataException("Did not find a Superblock.")
JLD2.InvalidDataException("Did not find a Superblock.")


# Print Single Assembly Path

In [54]:
@load "../../Data/hpc_out/rwm_wip_2_6r7m/79_rwm_wip_2_6r7m.jld2" input output


2-element Vector{Symbol}:
 :input
 :output

In [55]:
mindex = argmin(output["Es"])
lower = 500
upper = 100
out_folder = "../../Data/tmp/assembly_path/"
try
    rm(out_folder, recursive=true)
catch 
end
mkdir(out_folder)
radii = collect(Base.Iterators.flatten([input["template_radii"] for _ in 1:input["n_mol"]]))
for (i,j) in enumerate(maximum([mindex-lower,1]):minimum([mindex+upper, length(output["Es"])]))
    state = output["states"][j]
    MorphoMol.Utilities.state_to_poly(MorphoMol.Utilities.get_flat_realization(state, input["template_centers"]), radii, "$(out_folder)$(i)", input["n_mol"], length(input["template_radii"]))
end



# Dispersed State Energies

In [19]:
folder = "assets/input/poly/singles/"
for file in readdir(folder)
    c,r = MorphoMol.Utilities.poly_to_state("$folder$file")
    E = MorphoMol.Energies.solvation_free_energy(
        c, 
        length(r),
        r,
        1.4,
        MorphoMol.Energies.get_prefactors(1.4, 0.3665),
        0.0,
        0.85
        )
    println("$(split(file, "_")[1]) ", 2*E)
end

1ei7 8255.3356279267
2tmv 8130.426867538925
6r7m 8219.578304239454
6sae 8185.217724310747
6sag 8174.157936120334
